In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [3]:
#train/test/submission dataset load
train_data = pd.read_csv('/content/drive/MyDrive/데이콘/따릉이/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/데이콘/따릉이/test.csv')
submission_data = pd.read_csv('/content/drive/MyDrive/데이콘/따릉이/submission.csv')

# **1. Train/Test Split**


In [4]:
train_data.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [5]:
test_data.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
3,4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
4,5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0


In [29]:
#결측치 대체
#hour_bef_ozone, hour_bef_pm10, hour_bef_pm2.5는 결측치 대체 없이 나머지 변수들은 평균값으로 대체함.
train_data.isna().sum()

id                          0
hour                        0
hour_bef_temperature        0
hour_bef_precipitation      0
hour_bef_windspeed          0
hour_bef_humidity           0
hour_bef_visibility         0
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [31]:
#결측치 대체
#hour_bef_ozone, hour_bef_pm10, hour_bef_pm2.5는 결측치 대체 없이 나머지 변수들은 평균값으로 대체함.
test_data.isna().sum()

id                         0
hour                       0
hour_bef_temperature       0
hour_bef_precipitation     0
hour_bef_windspeed         0
hour_bef_humidity          0
hour_bef_visibility        0
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64

In [28]:
train_data['hour_bef_temperature'].fillna(value = train_data['hour_bef_temperature'].sum() / (len(train_data) - len(str(train_data['hour_bef_temperature'].isna().sum()))), inplace=True)
train_data['hour_bef_precipitation'].fillna(value = train_data['hour_bef_precipitation'].sum() / (len(train_data) - len(str(train_data['hour_bef_precipitation'].isna().sum()))), inplace=True)
train_data['hour_bef_windspeed'].fillna(value = train_data['hour_bef_windspeed'].sum() / (len(train_data) - len(str(train_data['hour_bef_windspeed'].isna().sum()))), inplace=True)
train_data['hour_bef_humidity'].fillna(value = train_data['hour_bef_humidity'].sum() / (len(train_data) - len(str(train_data['hour_bef_humidity'].isna().sum()))), inplace=True)
train_data['hour_bef_visibility'].fillna(value = train_data['hour_bef_visibility'].sum() / (len(train_data) - len(str(train_data['hour_bef_visibility'].isna().sum()))), inplace=True)

In [30]:
test_data['hour_bef_temperature'].fillna(value = test_data['hour_bef_temperature'].sum() / (len(test_data) - len(str(test_data['hour_bef_temperature'].isna().sum()))), inplace=True)
test_data['hour_bef_precipitation'].fillna(value = test_data['hour_bef_precipitation'].sum() / (len(test_data) - len(str(test_data['hour_bef_precipitation'].isna().sum()))), inplace=True)
test_data['hour_bef_windspeed'].fillna(value = test_data['hour_bef_windspeed'].sum() / (len(test_data) - len(str(test_data['hour_bef_windspeed'].isna().sum()))), inplace=True)
test_data['hour_bef_humidity'].fillna(value = test_data['hour_bef_humidity'].sum() / (len(test_data) - len(str(test_data['hour_bef_humidity'].isna().sum()))), inplace=True)
test_data['hour_bef_visibility'].fillna(value = test_data['hour_bef_visibility'].sum() / (len(test_data) - len(str(test_data['hour_bef_visibility'].isna().sum()))), inplace=True)

In [32]:
features_selection = ['hour', 'hour_bef_temperature', 'hour_bef_precipitation', 'hour_bef_humidity']
X_train = train_data[features_selection]
y_train = train_data['count']
X_test = test_data[features_selection]

In [33]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(1459, 4)
(1459,)
(715, 4)


# **2. Modeling**


## RandomForestRegressor parameters

(n_estimators: int = 100, *, criterion: str = "squared_error", max_depth: Any | None = None, min_samples_split: int = 2, min_samples_leaf: int = 1, min_weight_fraction_leaf: float = 0, max_features: float = 1, max_leaf_nodes: Any | None = None, min_impurity_decrease: float = 0, bootstrap: bool = True, oob_score: bool = False, n_jobs: Any | None = None, random_state: Any | None = None, verbose: int = 0, warm_start: bool = False, ccp_alpha: float = 0, max_samples: Any | None = None) -> None

In [34]:
model_Rfregressor = RandomForestRegressor(n_estimators=100, random_state=30)

In [35]:
model_Rfregressor.fit(X_train, y_train)

RandomForestRegressor(random_state=30)

In [36]:
ypred = model_Rfregressor.predict(X_test)

In [37]:
submission_data['count'] = ypred

In [38]:
submission_data

,id,count
0,0,90.520000
1,1,235.740000
2,2,69.580000
3,4,33.800000
4,5,88.540000
...,...,...
710,2148,67.077333
711,2149,68.100000
712,2165,122.380000
713,2166,162.640000


In [ ]:
#submission_data.to_csv('model.csv', index=False)